In [12]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [16]:
config = json.load(open('../config.json', 'r'))

In [17]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [18]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [22]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [23]:
fname = 'feed.xml'
url = 'http://dataskeptic.com/feed.rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

In [24]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [50]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [51]:
df.shape

(1, 7)

In [52]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [53]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    dest = emap[guid]
    response = urllib.request.urlopen(dest)
    data = response.read()
    fname = 'temp.mp3'
    f = open(fname, 'wb')
    f.write(data)
    f.close()
    audio = MP3(fname)
    duration = int(audio.info.length)
    q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
    r1 = conn.execute(q1)
    dest = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/ai-cover.png'
    q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
    r2 = conn.execute(q2)

In [54]:
#conn.execute("delete from related_content where blog_id=382")

In [55]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,659,382,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,The Experimental Design of Paranormal Claims,"In this episode of Data Skeptic, Kyle chats wi...",2018-05-04 16:36:31,NaN,-1.0
1,658,382,http://traffic.libsyn.com/dataskeptic/the-expe...,mp3,The Experimental Design of Paranormal Claims,"In this episode of Data Skeptic, Kyle chats wi...",2018-05-04 16:36:31,NaN,1651.0
2,653,370,,internal-link,Part 2,Listen to the next segment!,2018-05-03 00:03:34,371.0,NaN
3,652,380,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Hector Levesque,Hector Levesque holds a PhD in computer scienc...,2018-04-30 03:17:17,-1.0,NaN
4,651,381,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Table tennis robot trained with reinforcement ...,,2018-04-30 01:57:53,-1.0,NaN
5,650,381,,internal-link,AI Decision Making,An interview episode discussing reinforcement ...,2018-04-30 01:51:36,344.0,NaN
6,649,381,,internal-link,Reinforcement Learning,Our mini-episode on this topic.,2018-04-30 01:51:17,341.0,NaN
7,647,380,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Winograd Schema Challenge,"In this episode, Hector Levesque joins us to d...",2018-04-28 01:02:06,NaN,-1.0
8,646,380,http://traffic.libsyn.com/dataskeptic/winograd...,mp3,Winograd Schema Challenge,"In this episode, Hector Levesque joins us to d...",2018-04-28 01:02:06,NaN,2216.0
9,644,380,https://cs.nyu.edu/faculty/davise/papers/Winog...,external-link,Winograd Schemas,A compilation of Winograd schemas by Ernest Da...,2018-04-27 17:07:05,-1.0,NaN
